In [1]:
import numpy as np
import sys
import os
import re
import collections
import fnmatch
from scipy import misc
import matplotlib.pyplot as plt
from skimage import color
from skimage import io
from skimage.transform import resize, rescale, rotate, setup, warp, AffineTransform
from skimage.viewer import ImageViewer
from dim_reduce import pca_decmp
from sklearn.model_selection import train_test_split
from random import shuffle

from sklearn import svm
from sklearn import linear_model
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from matplotlib import pyplot as plt 
from visualize import plot
import pickle
from sklearn.decomposition import PCA 

/home/shyam/anaconda2/lib/python2.7/site-packages/skimage/viewer/utils/core.py:10: UserWarning: Recommended matplotlib backend is `Agg` for full skimage.viewer functionality.
  warn("Recommended matplotlib backend is `Agg` for full "


In [2]:
Male = ['AbrahamLincoln','AlbertEinstein','AamirKhan','BarackObama','AamirKhan','BhagatSingh','BillGates','BruceLee','CharlieChaplin','CheGuevara','DalaiLama','DanielCraig','DanielRadicliffe'
'DavidBeckham','DwayneJohnson','EmmaWatson','GeorgeClooney','HughJackman','JackieChan','JawaharlalNehru','JimCarrey','JohnLennon','JohnnyDepp','JustinBieber','KatrinaKaif'
'DiCaprio']
Female = ['AishwaryaRai','AngelinaJolie','BritneySpears','EmmaWatson','HillaryClinton','KatrinaKaif','LataMangeshkar','MarilynMonroe','MeganFox','MotherTeresa','OprahWinfrey',
'PrincessDiana','ScarlettJohansson','SelenaGomez','Shakira','TaylorSwift','AngelaMerkel','Beyonce','IndiraGandhi','J.K.Rowling','KateMiddleton','MargaretThatcher','NicoleKidman'
'QueenElizabeth']
File_path = 'cartoonFaces/'

In [3]:
CountM = 0
CountF = 0
MaxImg = 2500
Count_Img = 0
Label =np.zeros((2500),)
Data = np.zeros((2500,48,48),)
for file in os.listdir(File_path):
    if file.endswith(".jpeg"):
        FileStr = file       
        FileStr_ = FileStr[:len(FileStr)-9]
        if any(FileStr_ in s for s in Male):
            CountM = CountM + 1
            if CountM <= MaxImg/2:
                img = color.rgb2gray(io.imread(File_path + FileStr))
                img = resize(img, (48,48))
                Data[Count_Img,:,:] = img
                Label[Count_Img] = 1
                Count_Img = Count_Img + 1
        if any(FileStr_ in s for s in Female):
            CountF = CountF + 1
            if CountF <= MaxImg/2:
                img = color.rgb2gray(io.imread(File_path + FileStr))
                img = resize(img, (48,48))
                Data[Count_Img,:,:] = img
                Label[Count_Img] = -1
                Count_Img = Count_Img + 1

/home/shyam/anaconda2/lib/python2.7/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [4]:
Data = np.reshape(Data, (2500, 48*48))
X_train, X_test, y_train, y_test = train_test_split(Data,Label,test_size=0.2,random_state=43)

In [7]:
def Cartoon_SVM(X_train_pca, X_valid_pca,y_train, y_valid):
     clf = svm.SVC(C=0.1, kernel='linear',tol=0.001)
     clf.fit(X_train_pca, y_train)
     return (clf.predict(X_valid_pca))
    
def Cartoon_LR(X_train_pca, X_valid_pca,y_train, y_valid):
    logreg = linear_model.LogisticRegression(C=0.1)
    logreg.fit(X_train_pca, y_train)
    return (logreg.predict(X_valid_pca))

def Cartoon_bayes(X_train_pca, X_valid_pca,y_train, y_valid):
    clf = GaussianNB()
    clf.fit(X_train_pca, y_train)
    return (clf.predict(X_valid_pca))

def Cartoon_KNN(X_train_pca, X_valid_pca,y_train, y_valid):
    neigh = KNeighborsClassifier(n_neighbors=3)
    neigh.fit(X_train_pca, y_train)
    return (neigh.predict(X_valid_pca))

def Cartoon_DT(X_train_pca, X_valid_pca,y_train, y_valid):
     clf = DecisionTreeClassifier(random_state=0,max_depth=7)
     clf.fit(X_train_pca, y_train)
     return (clf.predict(X_valid_pca))

def Cartoon_Ensemble(X_train_pca, X_valid_pca,y_train, y_test):
    P_KNN = Cartoon_KNN(X_train_pca, X_test_pca,y_train, y_test)
    P_Bayes = Cartoon_bayes(X_train_pca, X_test_pca,y_train, y_test)
    P_LR = Cartoon_LR(X_train_pca, X_test_pca,y_train, y_test)
    P_SVM = Cartoon_SVM(X_train_pca, X_test_pca,y_train, y_test)
    P_DT = Cartoon_DT(X_train_pca, X_test_pca,y_train, y_test)
    return (np.sign(P_KNN+P_Bayes+P_LR+P_SVM+P_DT))

In [8]:
num_comp = [5,7,10,15,20,25,30,40,50]
#######################################Enseble Method ##############################################
num_comp = [25] #69.4
for comp in num_comp:
    X_train_pca, X_test_pca, eigenfaces = pca_decmp(X_train,X_test,comp)
    Prid_EM = Cartoon_Ensemble(X_train_pca, X_test_pca,y_train, y_test)
    print float(sum(Prid_EM == y_test))/len(y_test),comp

#######################################CartoonDT#####################################################
num_comp = [20] #63.6
for comp in num_comp:
    X_train_pca, X_test_pca, eigenfaces = pca_decmp(X_train,X_test,comp)
    Prid_DT = Cartoon_DT(X_train_pca, X_test_pca,y_train, y_test)
    print float(sum(Prid_DT == y_test))/len(y_test),comp
#####################################CartoonKNN#####################################################   
num_comp = [20] #65
for comp in num_comp:
    X_train_pca, X_test_pca, eigenfaces = pca_decmp(X_train,X_test,comp)
    Prid_KNN = Cartoon_KNN(X_train_pca, X_test_pca,y_train, y_test)
    print float(sum(Prid_KNN == y_test))/len(y_test),comp
    
#####################################CartoonNB#####################################################       
num_comp = [10] #65.4
for comp in num_comp:
    X_train_pca, X_test_pca, eigenfaces = pca_decmp(X_train,X_test,comp)
    Prid_NB = Cartoon_bayes(X_train_pca, X_test_pca,y_train, y_test)
    print float(sum(Prid_NB == y_test))/len(y_test),comp
    
#####################################CartoonLR#####################################################
num_comp = [100] #65.2
for comp in num_comp:
    X_train_pca, X_test_pca, eigenfaces = pca_decmp(X_train,X_test,comp)
    Prid_LR = Cartoon_LR(X_train_pca, X_test_pca,y_train, y_test)
    print float(sum(Prid_LR == y_test))/len(y_test),comp
    
#####################################CartoonSVM#####################################################    
num_comp = [20] #70
for comp in num_comp:
    X_train_pca, X_test_pca, eigenfaces = pca_decmp(X_train,X_test,comp)
    Prid_SVM = Cartoon_SVM(X_train_pca, X_test_pca,y_train, y_test)
    print float(sum(Prid_SVM == y_test))/len(y_test),comp

0.708 25
0.632 20
0.62 20
0.696 10
0.672 100
0.68 20


In [9]:
sh = list(range(len(y_test)))
shuffle(sh)
X_test_V = [np.reshape(X_test[sh[i]],(48,48)) for i in range(len(sh))]
print len(X_test)
y_ = [Prid_EM[sh[i]] for i in range(len(sh))]
plot(X_test_V, output="predictions.png", predictions=y_)

500


In [10]:
#pca = pickle.load(open('pca.pickle', 'rb'))
#X_test_pca = pca.transform(X_test)
#loaded_model = pickle.load(open('svm.pkl', 'rb'))
#result = loaded_model.score(X_test_pca, y_test)
#print(result)

In [11]:
File_path_real = 'realFaces/'
CountM = 0
CountF = 0
MaxImg = 250
Count_Img = 0
LabelR =np.zeros((250),)
DataR = np.zeros((250,48,48),)
for file in os.listdir(File_path_real):
    if file.endswith(".jpg"):
        FileStr = file       
        FileStr_ = FileStr[:len(FileStr)-9]
        if any(FileStr_ in s for s in Male):
            CountM = CountM + 1
            if CountM <= MaxImg/2:
                img = color.rgb2gray(io.imread(File_path_real + FileStr))
                img = resize(img, (48,48))
                DataR[Count_Img,:,:] = img
                LabelR[Count_Img] = 1
                Count_Img = Count_Img + 1
        if any(FileStr_ in s for s in Female):
            CountF = CountF + 1
            if CountF <= MaxImg/2:
                img = color.rgb2gray(io.imread(File_path_real + FileStr))
                img = resize(img, (48,48))
                DataR[Count_Img,:,:] = img
                LabelR[Count_Img] = -1
                Count_Img = Count_Img + 1

In [12]:
DataR = np.reshape(DataR, (250, 48*48))
X_valid, X_test, y_valid, y_test = train_test_split(DataR,LabelR,test_size=0.5,random_state=23)

In [13]:
#num_comp = [5,7,10,15,20,25,30,40,50]
#######################################Enseble Method ##############################################
num_comp = [30]
for comp in num_comp:
    X_train_pca, X_test_pca, eigenfaces = pca_decmp(X_train,X_test,comp)
    Prid_EM = Cartoon_Ensemble(X_train_pca, X_test_pca,y_train, y_valid)
    print (float(sum(Prid_EM == y_test))/len(y_test))*100

#######################################CartoonDT#####################################################
num_comp = [15]
for comp in num_comp:
    X_train_pca, X_test_pca, eigenfaces = pca_decmp(X_train,X_test,comp)
    Prid_DT = Cartoon_DT(X_train_pca, X_test_pca,y_train, y_test)
    print (float(sum(Prid_DT == y_test))/len(y_test))*100

#####################################CartoonKNN#####################################################   
num_comp = [20]
for comp in num_comp:
    X_train_pca, X_test_pca, eigenfaces = pca_decmp(X_train,X_test,comp)
    Prid_KNN = Cartoon_KNN(X_train_pca, X_test_pca,y_train, y_test)
    print (float(sum(Prid_KNN == y_test))/len(y_test))*100
    
#####################################CartoonNB#####################################################       
num_comp = [10]
for comp in num_comp:
    X_train_pca, X_test_pca, eigenfaces = pca_decmp(X_train,X_test,comp)
    Prid_NB = Cartoon_bayes(X_train_pca, X_test_pca,y_train, y_test)
    print (float(sum(Prid_NB == y_test))/len(y_test))*100
    
#####################################CartoonLR#####################################################
num_comp = [100]
for comp in num_comp:
    X_train_pca, X_test_pca, eigenfaces = pca_decmp(X_train,X_test,comp)
    Prid_LR = Cartoon_LR(X_train_pca, X_test_pca,y_train, y_test)
    print (float(sum(Prid_LR == y_test))/len(y_test))*100
    
#####################################CartoonSVM#####################################################    
num_comp = [20]
for comp in num_comp:
    X_train_pca, X_test_pca, eigenfaces = pca_decmp(X_train,X_test,comp)
    Prid_SVM = Cartoon_SVM(X_train_pca, X_test_pca,y_train, y_test)
    print (float(sum(Prid_SVM == y_test))/len(y_test))*100


64.0
64.0
56.0
60.8
61.6
62.4


In [14]:
sh = list(range(len(y_test)))
shuffle(sh)
X_test_V = [np.reshape(X_test[sh[i]],(48,48)) for i in range(len(sh))]
print len(X_test)
y_ = [Prid_EM[sh[i]] for i in range(len(sh))]
plot(X_test_V, output="predictions.png", predictions=y_)

125
